# Project 2: Analyzing Mental Health in the IT sector

This project explores mental health issues within the IT sector, using data from a survey conducted by [Open Source Mental Illness (OSMI)](https://osmihelp.org/) between 2014 and 2017. The survey examines attitudes towards mental health and the prevalence of mental health disorders in the IT field. A SQLite database containing this data is [available online](https://www.kaggle.com/datasets/anth7310/mental-health-in-the-tech-industry/data) and will be utilized in the project. Additionally, I formulate the following questions to answer based on the data:

1. Is there a relationship between gender and being diagnosed with a mental health disorder in the IT sector, based on survey data?
2. Does age correlate with seeking professional treatment for mental health issues?
3. Is there a connection between openess about mental health between employees from different countries?
4. How do mental health conditions affect productivity the IT field?

# Connecting to the database 

In [1]:
import sqlite3
import pandas as pd 

conn = sqlite3.connect('mental_health.sqlite')


Checking the names of the tables in the database

In [ ]:
# Create a cursor object
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

tables = cursor.fetchall()
for table in tables:
    table_name = table[0]  # Extract the table name 
    print(f" {table_name}")


In [ ]:
query = "SELECT * FROM Answer;"
df = pd.read_sql_query(query, conn)
display(df.head())


Checking the answers in the column 'AnswerText'

In [ ]:
print(df['AnswerText'].unique)


As the responses contain both numerical data (such as age and grade) and descriptive answers (such as Yes, No, Sometimes, Unsure), I will focus only on the data relevant to the questions I need to answer. For each question, I will inspect the AnswerText to identify the types of responses. After that, I will clean the data and remove Entries that are NaN, negative, or otherwise irrelevant.

I will also check the number of answers in each year

In [ ]:
import matplotlib.pyplot as plt  
import seaborn as sns
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# Count the number of answers for each year
answer_counts = df.groupby('SurveyID').size().reset_index(name='Count')

# Plot
sns.barplot(data=answer_counts, x='SurveyID', y='Count', palette='viridis')
plt.title('Number of Answers per Year',fontsize=14,weight='bold')
plt.xlabel('Year')
plt.ylabel('Number of Answers')
plt.xticks(rotation=0)
plt.show()

As shown in the figure above, the 2016 survey collected the highest number of answers. Therefore, I will focus my analysis on this year. It is worth noting that the 2015 survey is missing from the database. The reason for this is unclear, as I was not involved in its development.

Now that I have a better understanding of the data, I will join the Answer and Question tables. This will make it easier to visualize both the questions and answers. 

In [ ]:
def query_join_tables(conn):
    query = """
            SELECT q.QuestionID, q.QuestionText,  a.AnswerText,a.UserID, a.SurveyID
            FROM Question AS q
            RIGHT JOIN Answer AS a ON q.QuestionID = a.QuestionID
            WHERE a.SurveyID = 2016
            """
    return pd.read_sql_query(query,conn)

df_join_tables=query_join_tables(conn)
display(df_join_tables.head(5))

In the sequel, I will check the questions of the surveys. 

In [ ]:
print(df_join_tables[['questiontext', 'questionid']].drop_duplicates())

The methodology adopted to answer the questions is outlined as follows: first, read and understand the question; next, review the list of questions to identify the relevant ones; then, extract the necessary data, clean it, create visualizations, and, finally, draw conclusions.

# 1. Is there a relationship between gender and being diagnosed with a mental health disorder in the IT sector, based on survey data?

This analysis aims to examine the connection between gender and mental health diagnoses within the IT industry. Specifically, I seek to understand whether certain genders are more likely to experience mental health challenges.

In [ ]:
#'What is your gender? questionid?==2'
display(df_join_tables[df_join_tables['questionid'] == 2]['AnswerText'].unique())


To simplify my analyis, I will group the genders in tree majority groups (female, male, others)

In [ ]:
#'What is your gender? questionid?==2'
df_filtered = df_join_tables[
    (df_join_tables['questionid'] == 2)
][['questionid', 'questiontext', 'AnswerText', 'UserID']]
df_filtered['AnswerText'] = df_filtered['AnswerText'].apply(lambda x: x if x in ['Male', 'Female'] else 'Other')

# I will modify the terminology a bit
df_filtered['AnswerText'] = df_filtered['AnswerText'].apply(
    lambda x: 'Men' if x == 'Male' else 'Women' if x == 'Female' else 'Other'
)

#'Have you ever been diagnosed with a mental health disorder? questionid==34'
# I filtered the answers for Yes or No 
df_filtered2 = df_join_tables[
    (df_join_tables['questionid'] == 34) & 
    (df_join_tables['AnswerText'].isin(['Yes', 'No'])) 
][['questionid', 'questiontext', 'AnswerText', 'UserID']]

# I merged the data based on the UserID 
df_merged = pd.merge(df_filtered, df_filtered2, on='UserID', suffixes=('_gender', '_mental_health'))

display(df_merged.head())

In [ ]:
# Analyze the relationship by grouping by gender and mental health diagnosis
relationship = df_merged.groupby(['AnswerText_gender', 'AnswerText_mental_health']).size().reset_index(name='Count')

#Sort in ascending order
relationship = relationship.sort_values(by="Count", ascending=True)

# Create the plot
plt.figure(figsize=(10, 6))  
ax = sns.barplot(
    x='AnswerText_gender', 
    y="Count", 
    data=relationship, 
    hue='AnswerText_mental_health',  
    palette='viridis', 
    width=0.4
)

# Add title and labels
legend = ax.legend(title="Mental Health Diagnosis", title_fontsize=12, fontsize=10, loc='upper left', frameon=True)
legend.get_frame().set_edgecolor('black')
ax.set_title("Relationship Between Gender and Mental Health Disorder Diagnosis",fontsize=14, weight='bold')
ax.set_xlabel("Gender")
ax.set_ylabel("Number of answers")
plt.show()



The figure above indicates that the majority of participants in the survey were men, followed by women and other genders. It shows that more women reported having a mental health diagnosis compared to those without, which is also the case for other genders. In contrast, men predominantly reported not having a mental health diagnosis. 

# 2. Does age correlate with seeking professional treatment for mental health issues?


The second part of this project studies how age correlates with seeking professional treatment for mental health issues. This could inform organizations about how to approach mental health support across different age groups.

In [ ]:
# First, let's filter the data that we need to answer this questions
#  What is your age?, questionid=1
df_filtered_age = df_join_tables[
    (df_join_tables['questionid'] == 1)][['questionid', 'questiontext', 'AnswerText', 'UserID']]


# Now filter the data for questionid == 7
# Have you ever sought treatment for a mental health issue from a mental health professional?
# Let's check the answers for this question
print(df_join_tables[df_join_tables['questionid'] == 7]['AnswerText'].unique())



In [ ]:
# Filter the relevant data
df_filtered_treatment = df_join_tables[
    (df_join_tables['questionid'] == 7)
][['questionid', 'questiontext', 'AnswerText', 'UserID']]

# Map the 'AnswerText' column to 'Yes' for '1' and 'No' for '0' in the filtered data
df_filtered_treatment['AnswerText'] = df_filtered_treatment['AnswerText'].map({'1': 'Yes', '0': 'No'})

# Merge the data based on 'UserID'
df_merged_2 = pd.merge(df_filtered_age, df_filtered_treatment, on='UserID', suffixes=('_age', '_treatment'))

# Check the result
print(df_merged_2.head())


In [ ]:
# I will group the data by age groups
df_merged_2['AnswerText_age'] = pd.to_numeric(df_merged_2['AnswerText_age'], errors='coerce')

# Now apply the binning
bins = [18, 30, 40, 50, 60, 70]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69']
df_merged_2['agerange'] = pd.cut(df_merged_2['AnswerText_age'], bins, labels=labels, include_lowest=True)

# Check the result
display(df_merged_2[['AnswerText_age', 'agerange']].head())

In [ ]:
#Count of number of people by age group that answer Yes or No to the question about mental health treatment
age_group_treatment_count = df_merged_2.groupby('agerange')['AnswerText_treatment'].value_counts().unstack(fill_value=0)
age_group_treatment_count['Total'] = age_group_treatment_count.sum(axis=1)

print("Treatment Counts by Age Group:")
display(age_group_treatment_count)

As shown in the dataframe above, the 60-69 age group has a small number of answers (only 10), while the 18-29 and 30-39 age groups have the highest number of answers. Next, I will analyze the answers in terms of percentage

In [ ]:
#Here we will use a similar function, but this time I will compute the percentages
age_group_treatment = df_merged_2.groupby('agerange')['AnswerText_treatment'].value_counts(normalize=True).unstack(fill_value=0)
age_group_treatment_percentage = age_group_treatment * 100

#Plot
palette = sns.color_palette('viridis_r')
age_group_treatment_percentage.plot(kind='bar', stacked=True, color=palette, figsize=(10, 6))
plt.title('Percentage of Treatment Seekers by Age Group',fontsize=14, weight='bold')
plt.xlabel('Age Group')
plt.ylabel('Percentage (%)')
plt.xticks(rotation=0)
plt.legend(['Did Not Seek Treatment', 'Sought Treatment'], loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

Given that the 18-29 and 30-39 age groups have the highest number of responses, I will focus my analysis on these groups. The figure shows that a larger proportion of people in both age groups sought treatment for mental health issues. Unfortunately, the small number of answers in the other age groups may impact the reliability of my analysis, so I will exclude them.

# 3. Is there a significant difference in mental health challenges between employees from different countries or cultures?

Next, I examine the relationship between openness about mental health across different countries or cultures. Mental health awareness and attitudes vary globally, and cultural norms often influence how individuals approach mental health. By analyzing data from employees in different countries, the aim is to understand whether employees from certain cultures are more or less likely to discuss mental health openly.

In [ ]:
# Which country do you live in? questionid==3
df_filtered_country= df_join_tables[
(df_join_tables['questionid'] == 3)][['questionid', 'questiontext', 'AnswerText', 'UserID']]

# I will investigate the openess of the participants based on question 18
# Set option to display all columns and rows without truncation
pd.set_option('display.max_colwidth', None)  # No truncation for column content

# Print the specific row with questionid == 18
print(df_join_tables[df_join_tables['questionid'] == 18][['questionid', 'questiontext']].iloc[0])

# This questions asks the participants feel comfortable discussion mental health issues with co-workers
# I will use the term 'openness' to refer to this question.
df_filtered_openess = df_join_tables[
(df_join_tables['questionid'] == 18)& 
    (df_join_tables['AnswerText'].isin(['Yes', 'No', 'Maybe'])) 
][['questionid', 'questiontext', 'AnswerText', 'UserID']]

df_merged_3 = pd.merge(df_filtered_country, df_filtered_openess, on='UserID', suffixes=('_country', '_openess'))
relationship_3 = df_merged_3.groupby(['AnswerText_country', 'AnswerText_openess']).size().reset_index(name='Count')


In [20]:
# Grouping by 'AnswerText_country' and summing the counts
top_countries = relationship_3.groupby('AnswerText_country')['Count'].sum().sort_values(ascending=False).head(5)

# Now filter the relationship_3 dataframe for these top 5 countries
top_countries_data = relationship_3[relationship_3['AnswerText_country'].isin(top_countries.index)]

# Replace United Kingdom -> UK, United States of America -> USA
top_countries_data.loc[top_countries_data['AnswerText_country'] == 'United Kingdom', 'AnswerText_country'] = 'UK'
top_countries_data.loc[top_countries_data['AnswerText_country'] == 'United States of America', 'AnswerText_country'] = 'USA'
pivot_top_countries = top_countries_data.pivot(index='AnswerText_country', columns='AnswerText_openess', values='Count')


In [ ]:
total_counts = pivot_top_countries.sum(axis=1)
# Sort in ascending order for better readbility 
total_counts_sorted = total_counts.sort_values(ascending=True)

# Plotting the total counts for each country in a separate bar chart
plt.figure(figsize=(10, 6))
sns.barplot(x=total_counts_sorted.index, y=total_counts_sorted.values, palette='viridis_r',width=0.4)
plt.title('Total Number of Answers for Top 5 Countries', fontsize=14, weight='bold')
plt.xlabel('Country', fontsize=14)
plt.ylabel('Total Number of Answers', fontsize=14)
plt.xticks(rotation=0, ha='left', fontsize=12)
plt.show()

We can see that the answers of the survey are dominated by USA, followed by UK and Canada. 

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(20, 6))

# List to store colors and labels for the combined legend
legend_labels = pivot_top_countries.columns
legend_colors = sns.color_palette("crest", len(legend_labels))

for i, country in enumerate(pivot_top_countries.index):
    country_data = pivot_top_countries.loc[country]
    
    # Plot the pie chart for each country
    # 'autopct='%1.1f%%'' calculates and displays the percentage of each slice of the pie
    #  with one decimal place
    wedges, texts, autotexts = axes[i].pie(
        country_data, 
        autopct='%1.1f%%', 
        startangle=90, 
        colors=sns.color_palette("crest", len(country_data))
    )
    
    # Set the text color to white for better readbility 
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontsize(14) 
    
    # Add the title for each country or subplot
    axes[i].set_title(country, fontsize=14)

# Add a single legend for all pie charts (Yes, No, Maybe)
fig.legend(
    legend_labels, 
    loc="upper right", 
    fontsize=14, 
    ncol=len(legend_labels), 
    title='Answers',
    title_fontsize=14  
)

# Set the main title for the entire plot
plt.suptitle('Openness about mental health in the top 5 countries', fontsize=18, weight='bold', ha='center')
plt.tight_layout()
plt.show()

Overall, the responses indicate that participants do not feel comfortable sharing mental health issues with co-workers, as the percentage of "maybe" or "no" answers is larger than that of "yes." The exception is the Netherlands, where the percentages of "maybe" and "yes" responses are equal. However, the smaller number of responses from the Netherlands may limit the analysis. 

# 4. How do different medical conditions impact career performance in the IT field?

Next, I focus on understanding how various mental health conditions affect employees' career progress. I also examines whether specific conditions lead to particular challenges for IT professionals in terms of performance.








In [ ]:
#Filter data to answer the question

# If yes, what condition(s) have you been diagnosed? questionid=115
df_filtered_condition = df_join_tables[
    (df_join_tables['questionid'] == 115)
][['questionid', 'questiontext', 'AnswerText', 'UserID']]

display(df_filtered_condition.head())

In [24]:
df_filtered_condition = df_filtered_condition[df_filtered_condition['AnswerText'] != '-1']

In [ ]:
# How has it affected your career? question id=54
df_filtered_effect_career = df_join_tables[
    (df_join_tables['questionid'] == 54)][['questionid', 'questiontext', 'AnswerText', 'UserID']]

display(df_filtered_effect_career.head())

In [ ]:
df_filtered_effect_career = df_filtered_effect_career[df_filtered_effect_career['AnswerText'] != '-1']
display(df_filtered_effect_career.head())


In [ ]:
df_merged_4 = pd.merge(df_filtered_effect_career, df_filtered_condition, on='UserID', suffixes=('_prod', '_condition'))
display(df_merged_4.head())

In [ ]:
display(df_merged_4['AnswerText_condition'].unique)

In [ ]:
df_merged_4['GroupedCondition'] = df_merged_4['AnswerText_condition'].str.extract(r"^([\w\s]+ Disorder)")

# Check the number of NaN values in the column
print(f"Number of NaN values in 'GroupedCondition': {df_merged_4['GroupedCondition'].isna().sum()}")


In [ ]:
#Drop NaN rows 
df_merged_4 = df_merged_4.dropna(subset=['GroupedCondition'])
print(f"Number of NaN values in 'GroupedCondition': {df_merged_4['GroupedCondition'].isna().sum()}")


In [ ]:
display(df_merged_4.head())

In [32]:

disorder_productivity = df_merged_4.groupby(['GroupedCondition', 'AnswerText_prod']).size().reset_index(name='Count')

# Calculate the total count for each condition and filter the top 3
top_conditions = disorder_productivity.groupby('GroupedCondition')['Count'].sum().nlargest(3).index
top_disorder_productivity = disorder_productivity[disorder_productivity['GroupedCondition'].isin(top_conditions)]
top_disorder_productivity.loc[top_disorder_productivity['GroupedCondition'] == 'Attention Deficit Hyperactivity Disorder', 'GroupedCondition'] = 'ADHD'

In [ ]:
# Plot
plt.figure(figsize=(10, 6))
sns.barplot(
    data=top_disorder_productivity,
    x='GroupedCondition',
    y='Count',
    hue='AnswerText_prod',
    palette='viridis_r'
)

# Customize the plot
plt.title('Productivity Question Answers for Top 3 Conditions', fontsize=14, weight='bold')
plt.xlabel('Disorder', fontsize=12)
plt.ylabel('Number of Answers', fontsize=12)
plt.xticks(rotation=0, ha='left', fontsize=10)
plt.legend(title='Productivity Answer', fontsize=10)
plt.show()

Based on the high number of positive answers, we can see that participants strongly feel that their conditions (anxiety, ADHD, and mood disorders) affect their productivity at work.

# Conclusions 

This project was an interesting opportunity to practice my skills in data visualization, manipulation, and analysis. However, I felt that the small number of responses from the groups I was interested in may have limited the depth of the analysis. In addition, there were some issues with the answers. For example, many responses related to the disorders were quite similar, making it challenging to define the best way to group them. The gender options included a long list, and the answers to question 3 included '1' and '0,' whereas I was expecting "yes" or "no."

To improve the analysis in future projects, it would be helpful to gather responses from a larger, more specific group and standardize answer formats, especially for yes/no questions, to reduce ambiguity. 

In [109]:
conn.close()
